In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import warnings
warnings.filterwarnings('ignore')
from collections import Counter
from sklearn.model_selection import train_test_split
from scipy import sparse

In [2]:
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense , merge
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop,Nadam
from keras.utils import to_categorical
from keras.utils.vis_utils import model_to_dot
from keras.callbacks import ReduceLROnPlateau
from keras.layers.merge import dot
from keras.models import Model

Using TensorFlow backend.


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import OneHotEncoder, QuantileTransformer
from sklearn.model_selection import train_test_split
from numba import jit # Compile some functions when performance is critical
import keras
from keras.initializers import RandomNormal
from keras.models import Model, load_model, save_model
from keras.layers import Embedding, Input, Dense, Concatenate, Multiply, Flatten
from keras.optimizers import Adam
import tensorflow as tf

In [4]:
data1= pd.read_csv('Dataset/BookDetails.csv')

FileNotFoundError: [Errno 2] File b'Dataset/BookDetails.csv' does not exist: b'Dataset/BookDetails.csv'

In [ ]:
data1

In [ ]:
book = data1[['book_id','text_reviews_count','average_rating','publisher','publication_year',
             'ratings_count','genre1']]

In [ ]:
book

In [11]:
book['Pub1'] = book['publication_year']


In [12]:
book.iloc[1][7]

1995.0

In [15]:
book

,book_id,text_reviews_count,average_rating,publisher,publication_year,ratings_count,genre1,Pub1,PY1
0,10365,7868,4.04,Yearling,2000.0,271951,children,2000.0,
1,10959,4324,3.88,Phoenix,1995.0,111215,children,1995.0,
2,5,28561,4.53,Scholastic Inc.,2004.0,1876252,children,2004.0,
3,11387515,31536,4.43,Knopf,2012.0,255461,children_YoungAdult,2012.0,
4,438492,3313,3.91,HarperCollins Publishers,2000.0,121250,children,2000.0,
...,...,...,...,...,...,...,...,...,...
19288,25698127,140,4.01,Penguin Teen Australia,2016.0,533,YoungAdult,2016.0,
19289,13919896,48,3.58,HarperTeen,2014.0,177,YoungAdult,2014.0,
19290,8129464,229,4.07,Puffin,2010.0,3295,YoungAdult,2010.0,
19291,10860716,78,3.84,HarperCollins Canada,2011.0,258,YoungAdult,2011.0,


In [5]:
book["PY1"] = ""

NameError: name 'book' is not defined

In [ ]:
for i in range(len(book)):
    if book.iloc[i][7]>=2000.0:
        book.iat[i,8]='a'
    elif book.iloc[i][7]>=1950.0 and book.iloc[i][7]<2000.0:
        book.iat[i,8]='b'
    elif book.iloc[i][7]=="NaN":
        book.iat[i,8]=''
    else:
        book.iat[i,8]='d'
        
    
        

In [ ]:
book

In [ ]:
book

In [19]:
data1= pd.read_csv('Dataset/UserInteractionV1.csv')

In [20]:
data2 = data1[['user_id','book_id','rating']]

In [21]:
data2 = data2.loc[:, ~data2.columns.str.contains('^Unnamed')]

In [22]:
users = data2.user_id.unique()
Books = data2.book_id.unique()

userid2idx = {o:i for i,o in enumerate(users)}
Bookid2idx = {o:i for i,o in enumerate(Books)}

In [23]:
data2['user_id'] = data2['user_id'].apply(lambda x: userid2idx[x])
data2['book_id'] = data2['book_id'].apply(lambda x: Bookid2idx[x])

In [24]:
book['book_id'] = book['book_id'].apply(lambda x: Bookid2idx[x])
book

,book_id,text_reviews_count,average_rating,publisher,publication_year,ratings_count,genre1,Pub1,PY1
0,0,7868,4.04,Yearling,2000.0,271951,children,2000.0,a
1,1,4324,3.88,Phoenix,1995.0,111215,children,1995.0,b
2,2,28561,4.53,Scholastic Inc.,2004.0,1876252,children,2004.0,a
3,3,31536,4.43,Knopf,2012.0,255461,children_YoungAdult,2012.0,a
4,4,3313,3.91,HarperCollins Publishers,2000.0,121250,children,2000.0,a
...,...,...,...,...,...,...,...,...,...
19288,19288,140,4.01,Penguin Teen Australia,2016.0,533,YoungAdult,2016.0,a
19289,19289,48,3.58,HarperTeen,2014.0,177,YoungAdult,2014.0,a
19290,19290,229,4.07,Puffin,2010.0,3295,YoungAdult,2010.0,a
19291,19291,78,3.84,HarperCollins Canada,2011.0,258,YoungAdult,2011.0,a


In [25]:
data1

,Unnamed: 0,book_id,Unnamed: 0.1,user_id,timestamp,rating,has_spoiler
0,0,10365,45,8842281e1d1347389f2ab93d60773d4d,2017-03-22,5,False
1,1,10365,9550,55475556ab32863fed66cb94a404af71,2013-01-27,5,False
2,2,10365,44316,e323acfc7b38cab03e68fb74bc32da45,2015-05-28,5,False
3,3,10365,62420,b85e8348c1e1f6ed6ae8e76bb6de7f15,2011-05-20,5,False
4,4,10365,68963,df7068f85819b1d0bd6b4ed20096692d,2011-11-13,5,False
...,...,...,...,...,...,...,...
1114794,1114794,35383830,1048804,79addbe68a26cbd8baa27e3b067a7b25,2017-10-25,4,False
1114795,1114795,35383830,1059546,be9de9603d71beb5b964a80dc6fc4778,2017-08-17,4,True
1114796,1114796,35383830,1109274,06ca020a59e70d460b5644588fdb2c20,2017-07-22,5,False
1114797,1114797,35383830,1151507,8481f58b5d184a4cc6d362e390ab2079,2017-10-23,4,True


In [26]:
book['features_genre'] = book['genre1']

In [27]:
book['features_genre'] = book['features_genre'].map(lambda x: x.split('_'))

In [28]:
book

,book_id,text_reviews_count,average_rating,publisher,publication_year,ratings_count,genre1,Pub1,PY1,features_genre
0,0,7868,4.04,Yearling,2000.0,271951,children,2000.0,a,[children]
1,1,4324,3.88,Phoenix,1995.0,111215,children,1995.0,b,[children]
2,2,28561,4.53,Scholastic Inc.,2004.0,1876252,children,2004.0,a,[children]
3,3,31536,4.43,Knopf,2012.0,255461,children_YoungAdult,2012.0,a,"[children, YoungAdult]"
4,4,3313,3.91,HarperCollins Publishers,2000.0,121250,children,2000.0,a,[children]
...,...,...,...,...,...,...,...,...,...,...
19288,19288,140,4.01,Penguin Teen Australia,2016.0,533,YoungAdult,2016.0,a,[YoungAdult]
19289,19289,48,3.58,HarperTeen,2014.0,177,YoungAdult,2014.0,a,[YoungAdult]
19290,19290,229,4.07,Puffin,2010.0,3295,YoungAdult,2010.0,a,[YoungAdult]
19291,19291,78,3.84,HarperCollins Canada,2011.0,258,YoungAdult,2011.0,a,[YoungAdult]


In [29]:
book['PY1'] = book['PY1'].map(lambda x: x)

In [30]:
book

,book_id,text_reviews_count,average_rating,publisher,publication_year,ratings_count,genre1,Pub1,PY1,features_genre
0,0,7868,4.04,Yearling,2000.0,271951,children,2000.0,a,[children]
1,1,4324,3.88,Phoenix,1995.0,111215,children,1995.0,b,[children]
2,2,28561,4.53,Scholastic Inc.,2004.0,1876252,children,2004.0,a,[children]
3,3,31536,4.43,Knopf,2012.0,255461,children_YoungAdult,2012.0,a,"[children, YoungAdult]"
4,4,3313,3.91,HarperCollins Publishers,2000.0,121250,children,2000.0,a,[children]
...,...,...,...,...,...,...,...,...,...,...
19288,19288,140,4.01,Penguin Teen Australia,2016.0,533,YoungAdult,2016.0,a,[YoungAdult]
19289,19289,48,3.58,HarperTeen,2014.0,177,YoungAdult,2014.0,a,[YoungAdult]
19290,19290,229,4.07,Puffin,2010.0,3295,YoungAdult,2010.0,a,[YoungAdult]
19291,19291,78,3.84,HarperCollins Canada,2011.0,258,YoungAdult,2011.0,a,[YoungAdult]


In [31]:
all_genres = set()
for row in book['features_genre']:
    # Union of sets is declared with the | operator
    all_genres = all_genres | set(row)
#all_genres.remove('') # Drop the empty genre

def invert_dict(d):
    return {value: key for key, value in d.items()}

all_genres = sorted(list(all_genres)) # We convert it to a list to enforce alphabetic ordering
ngenres = len(all_genres)

idx2genre = dict(enumerate(all_genres)) # Create a mapping dictionary from index to dict
genre2idx = invert_dict(idx2genre) # Inverse dict

genre2idx

{'YoungAdult': 0,
 'biography': 1,
 'children': 2,
 'comics-graphic': 3,
 'crime': 4,
 'fantasy': 5,
 'history': 6,
 'mystery': 7,
 'paranormal': 8,
 'poetry': 9,
 'romance': 10,
 'thriller': 11}

In [32]:
all_year = set()
for row in book['PY1']:
    # Union of sets is declared with the | operator
    all_year = all_year | set(row)
#all_genres.remove('') # Drop the empty genre

def invert_dict(d):
    return {value: key for key, value in d.items()}

all_year = sorted(list(all_year)) # We convert it to a list to enforce alphabetic ordering
nyear = len(all_year)

idx2year= dict(enumerate(all_year)) # Create a mapping dictionary from index to dict
year2idx = invert_dict(idx2year) # Inverse dict

year2idx

{'a': 0, 'b': 1, 'd': 2}

In [33]:
def encode_genres(genres):
    out = np.zeros(ngenres)
    for genre in genres:
        if genre == '':
            pass
        else:
            out[genre2idx[genre]] = 1
    return out.tolist()
book['features_genre'] = book['features_genre'].map(encode_genres)
book['features_genre']

0        [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1        [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2        [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3        [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4        [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                               ...                        
19288    [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
19289    [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
19290    [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
19291    [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
19292    [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: features_genre, Length: 19293, dtype: object

In [34]:
def encode_year(years):
    out = np.zeros(nyear)
    for year in years:
        if year == '':
            pass
        else:
            out[year2idx[year]] = 1
    return out.tolist()
book['PY1'] = book['PY1'].map(encode_year)
book['PY1']

0        [1.0, 0.0, 0.0]
1        [0.0, 1.0, 0.0]
2        [1.0, 0.0, 0.0]
3        [1.0, 0.0, 0.0]
4        [1.0, 0.0, 0.0]
              ...       
19288    [1.0, 0.0, 0.0]
19289    [1.0, 0.0, 0.0]
19290    [1.0, 0.0, 0.0]
19291    [1.0, 0.0, 0.0]
19292    [1.0, 0.0, 0.0]
Name: PY1, Length: 19293, dtype: object

In [35]:
book

,book_id,text_reviews_count,average_rating,publisher,publication_year,ratings_count,genre1,Pub1,PY1,features_genre
0,0,7868,4.04,Yearling,2000.0,271951,children,2000.0,"[1.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,4324,3.88,Phoenix,1995.0,111215,children,1995.0,"[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,2,28561,4.53,Scholastic Inc.,2004.0,1876252,children,2004.0,"[1.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,3,31536,4.43,Knopf,2012.0,255461,children_YoungAdult,2012.0,"[1.0, 0.0, 0.0]","[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,4,3313,3.91,HarperCollins Publishers,2000.0,121250,children,2000.0,"[1.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...
19288,19288,140,4.01,Penguin Teen Australia,2016.0,533,YoungAdult,2016.0,"[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19289,19289,48,3.58,HarperTeen,2014.0,177,YoungAdult,2014.0,"[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19290,19290,229,4.07,Puffin,2010.0,3295,YoungAdult,2010.0,"[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19291,19291,78,3.84,HarperCollins Canada,2011.0,258,YoungAdult,2011.0,"[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
#n_feats = len(book['features_genre'].iloc[0])

In [ ]:
#n_feats

In [36]:
data2 = data2.join(book['features_genre'], on = 'book_id').dropna()

In [37]:
data2=data2.join(book['PY1'], on = 'book_id').dropna()

In [38]:
data=data2

In [39]:
data

,user_id,book_id,rating,features_genre,PY1
0,0,0,5,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]"
1,1,0,5,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]"
2,2,0,5,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]"
3,3,0,5,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]"
4,4,0,5,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]"
...,...,...,...,...,...
1114794,10800,19292,4,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]"
1114795,11917,19292,4,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]"
1114796,5572,19292,5,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]"
1114797,11417,19292,4,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]"


In [40]:
data['features'] = data['features_genre'] + data['PY1']

In [41]:
data

,user_id,book_id,rating,features_genre,PY1,features
0,0,0,5,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,0,5,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,2,0,5,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,3,0,5,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,4,0,5,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...
1114794,10800,19292,4,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1114795,11917,19292,4,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1114796,5572,19292,5,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1114797,11417,19292,4,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [42]:
n_feats = len(data['features'].iloc[0])

In [43]:
n_feats

15

In [44]:
data=data.drop(['features_genre', 'PY1'], axis=1)

In [45]:
data

,user_id,book_id,rating,features
0,0,0,5,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,0,5,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,2,0,5,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,3,0,5,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,4,0,5,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...
1114794,10800,19292,4,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1114795,11917,19292,4,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1114796,5572,19292,5,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1114797,11417,19292,4,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [46]:
from sklearn.utils import shuffle
data4=data

In [47]:
data=shuffle(data)

In [48]:
x0 = data['user_id'].to_numpy()
x1 =data['book_id'].to_numpy()

In [ ]:
x2 = np.stack(data['features'].to_numpy())

In [ ]:
y = data['rating'].to_numpy()

In [ ]:
def get_model(num_users, num_items, num_item_feats, mf_dim, layers=[64, 32, 16, 8]):
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')
    feats_input = Input(shape=(num_item_feats,), dtype='float32', name = 'feats_input')

    # User&Item Embeddings for Matrix Factorization
    MF_Embedding_User = Embedding(input_dim = num_users + 1, output_dim = mf_dim,
                                  name = 'user_embedding',
                                  embeddings_initializer = RandomNormal(stddev=0.001),
                                  input_length = 1)
    MF_Embedding_Item = Embedding(input_dim = num_items + 1, output_dim = mf_dim,
                                  name = 'item_embedding',
                                  embeddings_initializer = RandomNormal(stddev=0.001),
                                  input_length = 1)
    
    # User&Item Embeddings for MLP part
    MLP_Embedding_User = Embedding(input_dim = num_users + 1, output_dim = int(layers[0] / 2),
                                   name = 'mlp_embedding_user',
                                   embeddings_initializer = RandomNormal(stddev=0.001),
                                   input_length = 1)
    MLP_Embedding_Item = Embedding(input_dim = num_items + 1, output_dim = int(layers[0] / 2),
                                   name = 'mlp_embedding_item',
                                   embeddings_initializer = RandomNormal(stddev=0.001),
                                   input_length = 1) 
    
    mf_user_latent = Flatten()(MF_Embedding_User(user_input))
    mf_item_latent = Flatten()(MF_Embedding_Item(item_input))
    mf_vector =dot([mf_user_latent, mf_item_latent])
       

    # MLP part with item features
    mlp_user_latent = Flatten()(MLP_Embedding_User(user_input))
    mlp_item_latent = Flatten()(MLP_Embedding_Item(item_input))
    
    mlp_vector = Concatenate()([mlp_user_latent, mlp_item_latent, feats_input])
    for l in layers:
        layer = Dense(l, activation='relu')
        mlp_vector = layer(mlp_vector)

    # Concatenate MF and MLP parts
    predict_vector = Concatenate()([mf_vector, mlp_vector])
    
    prediction_1=Dense(8, activation = 'relu')(predict_vector)
    #prediction_2=Dense(2, activation = 'relu')(prediction_1)
    # Final prediction layer
    prediction = Dense(1, activation = 'relu',
                       kernel_initializer = 'lecun_uniform',
                       name = 'prediction')(prediction_1)
    
    model = Model(input = [user_input, item_input, feats_input], output = prediction)
    return model

In [ ]:
def getData(k,i):
    l=len(data4)
    l1=int(l/k)
    j=l1*(i+1)
    i=l1*i
    x0_val=x0[i:j]
    x1_val=x1[i:j]
    x2_val=x2[i:j]
    y_val=y[i:j]
    
    if i==0:
        x0_train=x0[j:l]
        x1_train=x1[j:l]
        x2_train=x2[j:l]
        y_train=y[j:l]
    elif i==k-1:
        x0_train=x0[0:i]
        x1_train=x1[0:i]
        x2_train=x2[0:i]
        y_train=y[0:i]
    else:
        x0_train=np.concatenate((x0[0:i],x0[j:l]))
        x1_train=np.concatenate((x1[0:i],x1[j:l]))
        x2_train=np.concatenate((x2[0:i],x2[j:l]))
        y_train=np.concatenate((y[0:i],y[j:l]))    
              
    x_train=[x0_train,x1_train,x2_train]
    x_val=[x0_val,x1_val,x2_val]
    
    return x_train,y_train,x_val,y_val

In [ ]:
learning_rate = 0.01
batch_size = 256
#n_epochs = 20
mf_dim = 15
layers = [32, 16, 8]
n_users=len(users)
n_book=len(Books)
n_folds=10

In [ ]:
def root_mean_squared_error(y_train, y_val):
        return K.sqrt(K.mean(K.square(y_val - y_train))) 

In [ ]:
for i in range(n_folds):
    print("Training on Fold: ",i+1)
    
    x_train,y_train,x_val,y_val=getData(n_folds,i)
    
    model = None
    model = get_model(n_users, n_book, n_feats, mf_dim, layers)
    model.compile(optimizer = 
                  Adam(lr = 0.001), loss =root_mean_squared_error)
    results = model.fit(x = x_train, y = y_train, validation_data = (x_val, y_val),
                 batch_size = batch_size, epochs = 3, verbose = True)
    print("Val Score: ", model.evaluate(x_val, y_val))

In [ ]:
learning_rate = 0.01
batch_size = 256
#n_epochs = 20
mf_dim = 5
layers = [64,32, 16, 8]
n_users=len(users)
n_book=len(Books)


In [ ]:
n_folds=5
for i in range(n_folds):
    print("Training on Fold: ",i+1)
    
    x_train,y_train,x_val,y_val=getData(n_folds,i)
    
    model = None
    model = get_model(n_users, n_book, n_feats, mf_dim, layers)
    model.compile(optimizer = 
                  Adam(lr = 0.001), loss =root_mean_squared_error)
    results = model.fit(x = x_train, y = y_train, validation_data = (x_val, y_val),
                 batch_size = batch_size, epochs = 3, verbose = True)
    print("Val Score: ", model.evaluate(x_val, y_val))

In [ ]:
learning_rate = 0.01
batch_size = 256
#n_epochs = 20
mf_dim = 10
layers = [64,32, 16, 8]
n_users=len(users)
n_book=len(Books)



In [ ]:
n_folds=5
for i in range(n_folds):
    print("Training on Fold: ",i+1)
    
    x_train,y_train,x_val,y_val=getData(n_folds,i)
    
    model = None
    model = get_model(n_users, n_book, n_feats, mf_dim, layers)
    model.compile(optimizer = 
                  Adam(lr = 0.001), loss ="MAE")
    results = model.fit(x = x_train, y = y_train, validation_data = (x_val, y_val),
                 batch_size = batch_size, epochs = 3, verbose = True)
    print("Val Score: ", model.evaluate(x_val, y_val))